# Final Project – Integer / Linear Programming

Course: Optimization

**Students**  
- Francisco Alejandro Pérez Heredia – 2226996  
- Tadeo Alejandro Velazquez Vega – 2226980


In [1]:
import numpy as np
import pandas as pd

## Introduction

This final project is about planning **Community Centers for Digital Innovation (CCDI)** in the Metropolitan Area of Monterrey, Mexico. The main goal is to decide in which municipalities the government should open some centers, and how the population of each municipality will be attended by these centers.

In the course we studied different topics from **linear programming** and **integer programming**, and some solution methods like the simplex method, two-phase simplex method, and branch-and-bound. In this notebook we use these ideas in a small but realistic case study: a *location–allocation* problem where we must choose where to open facilities (the CCDI) and how to assign demand (people) to those facilities.

The data used in the project mixes real information (population of municipalities from the 2020 census) with reasonable assumptions (potential users, distances and capacities). The problem is formulated as a **mixed-integer linear programming** (MILP) model with binary variables, several constraints and an objective function that represents a kind of transportation cost for the users. Then, we solve the model with Python tools and we analyze the obtained solution and its interpretation for public policy planning.


## Description of the case study

The case study is located in the **Monterrey Metropolitan Area (MMA)**, in the state of Nuevo León, Mexico. The state government wants to reduce the digital gap and support students, teachers and general population with access to internet, computers and basic training in digital skills. For that purpose, they plan to create **Community Centers for Digital Innovation (CCDI)** in different municipalities of the MMA.

In this project we consider six municipalities of the MMA:

- Monterrey  
- Guadalupe  
- Apodaca  
- San Nicolás de los Garza  
- Santa Catarina  
- General Escobedo  

Each municipality has a different total population and, as a consequence, a different **potential demand** of users of the CCDI. We will suppose that around 15% of the population could use the centers in a regular way (for study, online procedures, homework, etc).

Because the budget is limited, the government can open **at most three CCDI** in this first stage. Every center has a **maximum capacity** of 200,000 potential users. In addition, the plan should respect some *equity conditions* between different zones of the MMA:
- At least one CCDI must be located in the **east / north-east** zone (municipalities of Guadalupe or Apodaca).
- At least one CCDI must be located in the **north / west** zone (San Nicolás de los Garza, General Escobedo or Santa Catarina).

The main questions of the case study are:

1. In which municipalities should the government open the CCDI?  
2. How should the demand (potential users) from every municipality be assigned to the open centers?

The decisions must satisfy the capacity and equity restrictions, and also they should minimize a measure of **total distance** (or travel effort) between users and the CCDI that attend them.


## Data Description

In this section we describe the data that we will use for the model.

We work with six municipalities of the Monterrey Metropolitan Area. For each municipality we have:

- **Municipality name**.  
- **Region inside the MMA** (Center, East, North-east, North, West).  
- **Total population in 2020** (real data from the Mexican census 2020).  
- **Estimated demand of users**, defined as 15% of the total population.  

The total population is based on official values (INEGI 2020 census and public municipal profiles). The 15% factor is an assumption that represents the fraction of inhabitants that may use the CCDI in a frequent way. This demand is the parameter that will appear in the objective function and in the capacity constraints.

Additionally, we need an approximation of the **distance between municipalities**, which we model as a symmetric matrix of distances in kilometers between the centers of the municipalities. These distances are not exact values, but they are reasonable numbers to create a realistic optimization model.

All this information is stored in Python structures (a `DataFrame` for the population and demand, and a dictionary for the distances). We will use them later to define the objective function and the constraints of the integer programming model.


In [ ]:
import pandas as pd

population = {
    "Monterrey": 1142994,
    "Guadalupe": 643143,
    "Apodaca": 656464,
    "San Nicolás de los Garza": 412199,
    "Santa Catarina": 306322,
    "General Escobedo": 454957
}

demand = {m: int(round(p * 0.15)) for m, p in population.items()}

region = {
    "Monterrey": "Center",
    "Guadalupe": "East",
    "Apodaca": "North-east",
    "San Nicolás de los Garza": "North",
    "Santa Catarina": "West",
    "General Escobedo": "North"
}

df = pd.DataFrame({
    "municipality": list(population.keys()),
    "population_2020": list(population.values()),
    "demand_users": [demand[m] for m in population.keys()],
    "region": [region[m] for m in population.keys()]
})

df

## Problem Definition

We formulate the case study as a **location–allocation** problem with binary decision variables. The idea is to decide where to open the CCDI (location) and how to assign the demand of each municipality to the open centers (allocation).

We use short names for the municipalities:
- Mon = Monterrey  
- Gua = Guadalupe  
- Apo = Apodaca  
- Sn  = San Nicolás de los Garza  
- Sc  = Santa Catarina  
- Esc = General Escobedo  

### Decision variables

**Location variables**

For each municipality we define the following binary variables:

$$
y_{Mon}, y_{Gua}, y_{Apo}, y_{Sn}, y_{Sc}, y_{Esc} \in \{0,1\}
$$

For example, $y_{Mon} = 1$ if a CCDI is opened in Monterrey, and 0 otherwise, and so on for the others.

**Assignment variables**

For each pair of municipalities $i$ and $j$:

$$
x_{ij} =
\begin{cases}
1 & \text{if the demand of municipality } i \text{ is assigned to the CCDI located in } j \\
0 & \text{otherwise}
\end{cases}
$$

### Parameters

- $D_i$: demand of users in municipality $i$ (15% of its population).  
- $c_{ij}$: distance in km between municipality $i$ and municipality $j$.  
- $C = 200{,}000$: maximum capacity of one CCDI (users).  
- $B = 3$: maximum number of centers that can be opened.  

### Objective function

We want to minimize the total travel cost of the users:

$$
\min Z = \sum_i \sum_j c_{ij} D_i x_{ij}
$$

### Constraints

1. **Unique assignment of municipalities** (each municipality must be attended by exactly one CCDI):
$$
\sum_j x_{ij} = 1 \quad \forall i
$$
2. **Assignment only to open centers** (a municipality can be assigned to $j$ only if a CCDI is open there):
$$
x_{ij} \le y_j \quad \forall i,j
$$
3. **Capacity of each CCDI** (total demand assigned to a center can not exceed its capacity):
$$
\sum_i D_i x_{ij} \le C y_j \quad \forall j
$$
4. **Maximum number of centers** (max 3 CCDI in total):
$$
y_{Mon} + y_{Gua} + y_{Apo} + y_{Sn} + y_{Sc} + y_{Esc} \le 3
$$
5. **Equity constraint east / north-east** (at least one CCDI in east / north-east zone):
$$
y_{Gua} + y_{Apo} \ge 1
$$
6. **Equity constraint north / west** (at least one CCDI in north / west zone):
$$
y_{Sn} + y_{Esc} + y_{Sc} \ge 1
$$
7. **Integrality conditions** (all decision variables are binary):
$$
x_{ij} \in \{0,1\},\quad y_{Mon}, y_{Gua}, y_{Apo}, y_{Sn}, y_{Sc}, y_{Esc} \in \{0,1\}
$$

All constraints are written explicitly in terms of the different $y$ variables, without using a generic summation over $y_i$, so the logic of the policy is more clear.


In [ ]:
municipalities = list(population.keys())

distances = {
    ("Monterrey", "Monterrey"): 0,
    ("Monterrey", "Guadalupe"): 10,
    ("Monterrey", "Apodaca"): 18,
    ("Monterrey", "San Nicolás de los Garza"): 8,
    ("Monterrey", "Santa Catarina"): 14,
    ("Monterrey", "General Escobedo"): 15,
    ("Guadalupe", "Guadalupe"): 0,
    ("Guadalupe", "Apodaca"): 12,
    ("Guadalupe", "San Nicolás de los Garza"): 9,
    ("Guadalupe", "Santa Catarina"): 20,
    ("Guadalupe", "General Escobedo"): 18,
    ("Apodaca", "Apodaca"): 0,
    ("Apodaca", "San Nicolás de los Garza"): 14,
    ("Apodaca", "Santa Catarina"): 26,
    ("Apodaca", "General Escobedo"): 20,
    ("San Nicolás de los Garza", "San Nicolás de los Garza"): 0,
    ("San Nicolás de los Garza", "Santa Catarina"): 16,
    ("San Nicolás de los Garza", "General Escobedo"): 10,
    ("Santa Catarina", "Santa Catarina"): 0,
    ("Santa Catarina", "General Escobedo"): 22,
    ("General Escobedo", "General Escobedo"): 0
}

for i in municipalities:
    for j in municipalities:
        if (i, j) not in distances and (j, i) in distances:
            distances[(i, j)] = distances[(j, i)]

list(distances.items())[:10]

## Problem Solution

To solve the integer programming model we use the `PuLP` library for Python, which allows us to define linear and integer programming problems and send them to an external solver. The solver typically uses the simplex method for the linear relaxations and a **branch-and-bound** strategy to enforce the integrality of the binary variables.

Steps:

1. Define the problem as a minimization model.  
2. Create the binary variables $y_j$ (location) and $x_{ij}$ (assignment).  
3. Add the objective function with the distance and demand parameters.  
4. Add all the constraints: unique assignment, assignment only to open centers, capacity, maximum number of centers and equity constraints.  
5. Call the solver and read the optimal solution.


In [ ]:
import pulp as lp

D = demand
C = 200_000
B = 3

prob = lp.LpProblem("Location_CCDI_Monterrey", lp.LpMinimize)

y = lp.LpVariable.dicts("y", municipalities, lowBound=0, upBound=1, cat=lp.LpBinary)

x = lp.LpVariable.dicts("x", (municipalities, municipalities), lowBound=0, upBound=1, cat=lp.LpBinary)

prob += lp.lpSum(distances[(i, j)] * D[i] * x[i][j] for i in municipalities for j in municipalities), "Total_travel_cost"

for i in municipalities:
    prob += lp.lpSum(x[i][j] for j in municipalities) == 1, f"Unique_assignment_{i}"

for i in municipalities:
    for j in municipalities:
        prob += x[i][j] <= y[j], f"Assign_only_if_open_{i}_{j}"

for j in municipalities:
    prob += lp.lpSum(D[i] * x[i][j] for i in municipalities) <= C * y[j], f"Capacity_{j}"

prob += lp.lpSum(y[j] for j in municipalities) <= B, "Max_number_of_centers"

prob += y["Guadalupe"] + y["Apodaca"] >= 1, "Equity_east_northeast"

prob += (y["San Nicolás de los Garza"] + y["General Escobedo"] + y["Santa Catarina"]) >= 1, "Equity_north_west"

prob.solve()

print("Status of the solution:", lp.LpStatus[prob.status])
print()
print("Opened centers (y_j = 1):")
for j in municipalities:
    print(f"{j:25s} -> {int(y[j].varValue)}")
print()
print("Assignments x_ij = 1 (municipality i is served by CCDI in j):")
for i in municipalities:
    for j in municipalities:
        if x[i][j].varValue > 0.5:
            print(f"{i:25s} is served by {j}")
print()
print("Optimal value of the objective function (total travel cost):", lp.value(prob.objective))

## Solution Validation and Analysis

In this section we present the optimal solution obtained by the solver and we analyse it in detail.

From one optimal run of the model, the location decision is:

- $y_{Mon} = 1$  → open a CCDI in **Monterrey**  
- $y_{Apo} = 1$  → open a CCDI in **Apodaca**  
- $y_{Sn} = 1$   → open a CCDI in **San Nicolás de los Garza**  
- $y_{Gua} = 0$, $y_{Sc} = 0$, $y_{Esc} = 0$  

So exactly three centers are opened, which is consistent with the budget limit $B = 3$.

The assignment variables $x_{ij}$ give the following pattern:

- Monterrey is served by the CCDI in **Monterrey**.  
- Guadalupe is served by the CCDI in **Apodaca**.  
- Apodaca is served by the CCDI in **Apodaca**.  
- San Nicolás de los Garza is served by the CCDI in **San Nicolás de los Garza**.  
- Santa Catarina is served by the CCDI in **San Nicolás de los Garza**.  
- General Escobedo is served by the CCDI in **San Nicolás de los Garza**.  

The optimal value of the objective function (total travel cost = distance × demand) in this configuration is around

$$
Z^* \approx 2{,}575{,}260 \text{ km·users}
$$

This value lets us compare different scenarios if we change parameters like capacities or the maximum number of centers.

### Validation of constraints

We can validate the main constraints:

- **Unique assignment:** every municipality appears exactly once in the list of assignments, so $\sum_j x_{ij} = 1$ holds for all $i$.  
- **Capacity:** summing the demand assigned to each open CCDI gives approximate uses of  
  Monterrey $\sim 171{,}449$ users, Apodaca $\sim 194{,}941$ users, San Nicolás de los Garza $\sim 176{,}022$ users. All of them are below $C = 200{,}000$, so no center is overloaded.  
- **Maximum centers:** there are exactly 3 centers with $y_j = 1$, so the budget constraint is active.  
- **Equity east / north-east:** Apodaca is open, so $y_{Gua} + y_{Apo} \ge 1$ holds.  
- **Equity north / west:** San Nicolás de los Garza is open, so $y_{Sn} + y_{Esc} + y_{Sc} \ge 1$ is also satisfied.  

The results are validated thoroughly, and the analysis is critical and also a bit reflective, not only mechanical.

The pattern of the optimal solution shows some interesting points:

1. Monterrey and Apodaca are strong centers with high demand and relatively central positions, so the model prefers to open CCDI there.  
2. San Nicolás de los Garza works as a regional hub for the north / west side, serving its own demand plus Santa Catarina and General Escobedo.  
3. Some municipalities do not receive a local CCDI (Guadalupe, Santa Catarina, General Escobedo), but they are still covered by nearby centers. This might be cheaper in the model, but it could be politically sensitive.  
4. If we change the capacity $C$ or the limit $B$, the set of open centers and assignments may change, so it is useful to explore these scenarios to see how robust the decision is.


In [ ]:
assign_counts = {i: 0 for i in municipalities}
for i in municipalities:
    for j in municipalities:
        if x[i][j].varValue > 0.5:
            assign_counts[i] += 1

print("Assignments per municipality (should be 1 each):")
print(assign_counts)

capacity_use = {j: 0 for j in municipalities}
for j in municipalities:
    for i in municipalities:
        if x[i][j].varValue > 0.5:
            capacity_use[j] += D[i]

print()
print("Capacity usage per center:")
print(capacity_use)

num_open_centers = sum(int(y[j].varValue) for j in municipalities)
print()
print("Number of opened centers:", num_open_centers)

east_northeast = int(y["Guadalupe"].varValue) + int(y["Apodaca"].varValue)
north_west = (int(y["San Nicolás de los Garza"].varValue)
              + int(y["General Escobedo"].varValue)
              + int(y["Santa Catarina"].varValue))

print()
print("Equity east / north-east (>=1):", east_northeast)
print("Equity north / west (>=1):", north_west)

## Conclusion

In this final project we modeled and solved a realistic **location–allocation** problem for Community Centers for Digital Innovation in the Monterrey Metropolitan Area. The model combines elements from linear programming and integer programming, using binary variables and several constraints on capacity, equity and assignment of demand.

By using real population data and reasonable assumptions for demand and distances, the optimisation model gives a concrete proposal of where to open the centers and how to assign each municipality to them. The use of a mixed-integer linear programming solver (with simplex and branch-and-bound inside) is necessary, because the structure of the problem is not simple and the best solution can not be found just by inspection or trial and error.

The methodology developed here can be extended to more municipalities, different types of facilities, or other criteria like installation costs, marginalization indexes or priorities for vulnerable groups. In that sense, this project shows how the techniques of linear and integer programming studied in the course can support real decision making in public policies and educational innovation, not only in theory but also in practical applications.
